In [ ]:
import os
import sys

sys.path.insert(0, os.path.abspath("../.."))
from dotenv import load_dotenv

load_dotenv("../../.env", override=True)

from examples.config import settings

%load_ext autoreload
%autoreload 2

# AI Ideation Agent

This notebook demonstrates an **orchestrating AI ideation agent** using the `deepagents` package.

## Deep Agent Pattern Demonstrated

1. **PLANNING**: Uses `write_todos` to plan the ideation session before starting
2. **GUIDING**: Leads users through a 9-section AI Use Case Canvas conversationally
3. **DELEGATING**: Uses the `task` tool to delegate compliance analysis to a specialist subagent
4. **SYNTHESIZING**: Combines all inputs into exportable deliverables (Markdown + Jira Epic)

## Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    ORCHESTRATOR                              │
│                                                              │
│  Canvas Tools: start_new_canvas, get_section, save_answer,  │
│                save_section_answers, get_canvas_progress,   │
│                generate_canvas_export, etc.                 │
│                                                              │
│  Native Tools: write_todos, task, write_file, read_file     │
│                                                              │
│  State: Persisted to output/current_canvas.json             │
│         (NOT in context window - shared across agents)      │
│                                                              │
└──────────────────────┬──────────────────────────────────────┘
                       │ task(subagent_type="compliance-advisor")
                       ▼
┌─────────────────────────────────────────────────────────────┐
│                 SUBAGENT: compliance-advisor                 │
│                                                              │
│  Tools: assess_compliance_flags (reads same JSON file)      │
│  Expertise: EU AI Act, GDPR, IT Security                    │
└─────────────────────────────────────────────────────────────┘
```

## Import the Tools

The tools are organized by workflow phase.

In [ ]:
from examples.ai_ideation_agent.tools.tools import (
    # Initialization
    start_new_canvas,
    # Navigation
    get_canvas_overview,
    get_section,
    get_question,
    # Saving (single or batch)
    save_answer,
    save_section_answers,
    # Progress & Review
    get_canvas_progress,
    get_canvas_answers,
    # Compliance & Export
    assess_compliance_flags,
    generate_canvas_export,
)

tools = [
    start_new_canvas,
    get_canvas_overview,
    get_section,
    get_question,
    save_answer,
    save_section_answers,
    get_canvas_progress,
    get_canvas_answers,
    assess_compliance_flags,
    generate_canvas_export,
]

print("Canvas Tools (10):")
print("=" * 50)
print("INIT:     start_new_canvas")
print("NAVIGATE: get_canvas_overview, get_section, get_question")
print("SAVE:     save_answer, save_section_answers (batch)")
print("REVIEW:   get_canvas_progress, get_canvas_answers")
print("EXPORT:   assess_compliance_flags, generate_canvas_export")
print()
print("Native Deepagent Tools (auto-added):")
print("=" * 50)
print("PLAN:     write_todos")
print("DELEGATE: task (to subagents)")
print("FILES:    write_file, read_file")

## Creating the Agent

Create the AI ideation orchestrator with tools and compliance-advisor subagent.

In [ ]:
from deepagents import create_deep_agent
from langchain_google_genai import ChatGoogleGenerativeAI

from examples.ai_ideation_agent.prompts.prompts import (
    AI_IDEATION_AGENT_INSTRUCTIONS,
    COMPLIANCE_ADVISOR_INSTRUCTIONS,
)

model = ChatGoogleGenerativeAI(
    model=settings.GOOGLE_MODEL_NAME,
    temperature=settings.TEMPERATURE,
    google_api_key=settings.GOOGLE_API_KEY,
)

# Compliance advisor subagent - delegated for regulatory analysis
compliance_advisor_subagent = {
    "name": "compliance-advisor",
    "model": model,
    "description": (
        "Delegate compliance analysis to this specialist AFTER completing sections 6-8. "
        "The advisor analyzes EU AI Act risk classification, GDPR implications, "
        "and IT security requirements, providing detailed guidance and recommendations."
    ),
    "system_prompt": COMPLIANCE_ADVISOR_INSTRUCTIONS,
    "tools": [assess_compliance_flags],  # Reads from shared canvas file
}

# Create the orchestrator agent
agent = create_deep_agent(
    model=model,
    tools=tools,
    system_prompt=AI_IDEATION_AGENT_INSTRUCTIONS,
    subagents=[compliance_advisor_subagent],
)

print("AI Ideation Orchestrator created!")
print("=" * 50)
print(f"  Canvas tools: {len(tools)}")
print(f"  Native tools: write_todos, task, write_file, read_file")
print(f"  Subagents: compliance-advisor")
print(f"  State: output/current_canvas.json (file-based, not context)")

## Visualize the Agent Graph

In [ ]:
from IPython.display import Image, display

display(Image(agent.get_graph().draw_mermaid_png()))

## Clean Up Previous Demo Files

In [ ]:
from pathlib import Path

output_dir = Path("output")
if output_dir.exists():
    for f in output_dir.glob("*.md"):
        f.unlink()
    for f in output_dir.glob("*.json"):
        f.unlink()
    print("Previous demo files cleaned up.")
else:
    output_dir.mkdir(exist_ok=True)
    print("Output directory created.")

## Test the Orchestrator

Watch for:
1. **Planning** with write_todos
2. **Canvas initialization** with start_new_canvas
3. **Batch saving** with save_section_answers
4. **Delegation** to compliance-advisor via task

In [ ]:
from examples.utils import format_messages

# Test 1: Start a new ideation session
print("=" * 80)
print("TEST 1: Start ideation - Expect write_todos THEN start_new_canvas")
print("=" * 80)

result1 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "I have an idea for an AI solution: a document classification system that automatically categorizes incoming support tickets to route them to the right team.",
            }
        ],
    },
)
format_messages(result1["messages"])

In [ ]:
# Test 2: Provide stakeholder information (batch save opportunity)
print("\n" + "=" * 80)
print("TEST 2: Stakeholders - Expect save_section_answers (batch) or multiple save_answer")
print("=" * 80)

result2 = agent.invoke(
    {
        "messages": result1["messages"]
        + [
            {
                "role": "user",
                "content": """Here are all the stakeholders:
- Problem owner: Lisa Chen, Head of Customer Support
- Budget owner: David Park, CFO
- Technical owner: Raj Patel, Engineering Manager
- Executive sponsor: Maria Santos, COO""",
            }
        ],
    },
)
format_messages(result2["messages"][-5:])

In [ ]:
# Test 3: Check progress
print("\n" + "=" * 80)
print("TEST 3: Check progress - Shows section-by-section completion")
print("=" * 80)

result3 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Show me the current progress.",
            }
        ],
    },
)
format_messages(result3["messages"])

In [ ]:
# Test 4: Ask about compliance - should delegate to subagent
print("\n" + "=" * 80)
print("TEST 4: Compliance question - Expect task(compliance-advisor)")
print("=" * 80)

result4 = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Is a ticket classification system that processes customer messages considered high-risk under the EU AI Act? Give me detailed compliance guidance.",
            }
        ],
    },
)
format_messages(result4["messages"])

## View Generated Files

In [ ]:
output_dir = Path("output")
print("Generated files:")
print("=" * 40)
for f in sorted(output_dir.iterdir()):
    if not f.name.startswith("."):
        print(f"- {f.name}")

In [ ]:
import json

# Display the current canvas state (file-based persistence)
canvas_file = output_dir / "current_canvas.json"
if canvas_file.exists():
    print("Canvas State (from file, not context window):")
    print("=" * 40)
    data = json.loads(canvas_file.read_text())
    print(f"Use Case: {data.get('use_case_name')}")
    print(f"Status: {data.get('status')}")
    print(f"Answers: {len(data.get('answers', {}))} questions answered")
    print()
    print("Answers:")
    for qid, answer_data in data.get('answers', {}).items():
        print(f"  {qid}: {answer_data['answer'][:50]}...")

## Summary

### Tool Design Principles

| Principle | Implementation |
|-----------|----------------|
| Clear naming | `get_canvas_overview` not `load_canvas_template` |
| Single responsibility | `save_answer` (one) vs `save_section_answers` (batch) |
| LLM-friendly | `get_question(Q3.1)` for drilling into specifics |
| No context bloat | State in JSON file, not conversation history |
| Coherent with native | Custom tools complement `write_todos`, `task`, etc. |

### Tool Organization

```
Custom Tools (10)              Native Tools (4)
─────────────────              ────────────────
INIT:     start_new_canvas     PLAN:     write_todos
NAVIGATE: get_canvas_overview  DELEGATE: task
          get_section          FILES:    write_file
          get_question                   read_file
SAVE:     save_answer
          save_section_answers
REVIEW:   get_canvas_progress
          get_canvas_answers
EXPORT:   assess_compliance_flags
          generate_canvas_export
```

### State Management

All 32 question answers are stored in `output/current_canvas.json`, NOT in the context window:
- Keeps context lean for long conversations
- Allows both orchestrator and subagent to access same data
- Persists across sessions